**Import and Load API**

In [8]:
#Install yelpapi
!pip install yelpapi

In [9]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [10]:
#Install tmdbsimple (only need to run once)
!pip install tqdm

In [11]:
#Load API Credentials
with open('/Users/benjaminengel/Documents/.Secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [12]:
#Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [13]:
#Set my API call parameters 
LOCATION = 'Philadelphia,PA'
TERM = 'Cheesesteak'

In [14]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Philly_cheesesteak.json"
JSON_FILE

'Data/results_in_progress_Philly_cheesesteak.json'

In [15]:
#Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
##If it does not exist: 
if file_exists == False:
    
    ##CREATE ANY NEEDED FOLDERS
    #Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ##If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ##INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    #Save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
#If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Philly_cheesesteak.json not found. Saving empty list to file.


In [16]:
#Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
#Set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [17]:
#Use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [18]:
#How many results total?
total_results = results['total']
total_results

3700

In [19]:
#How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page
#20 per page

20

In [20]:
#Import additional packages for controlling our loop
import time, math
#Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

185

In [21]:
#Join new results with old list with extend and save it to a file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

**For Loop**

In [22]:
#TQDM for loop 
# We will also use time.sleep when executing API calls so that we do not overwhelm the server
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/185 [00:00<?, ?it/s]

In [23]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    #Read in results in progress file
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    #Save number of results
    n_results = len(previous_results)
    #Use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    #Append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    #Add a 200ms pause
    time.sleep(.2)
    
    #Too many results results for the free version 

  0%|          | 0/185 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

**Too many results for the basic/free version.**

In [24]:
#Delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [25]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    #Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ##If it DOES exist:
    if file_exists == True:
        
        #Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    #If it does NOT exist:
    else:
        
        #INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ##CREATE ANY NEEDED FOLDERS
        #Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        #If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ##Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [26]:
#Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
##Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
##Set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
#Use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
##How many results total?
total_results = results['total']
##How many did we get the details for?
results_per_page = len(results['businesses'])
#Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_Philly_cheesesteak.json not found. Saving empty list to new file.
- 0 previous results found.


185

In [27]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ##Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ##Save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ##Use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ##Append new results and save to file
    previous_results.extend(results['businesses'])
    
    #Display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/185 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [28]:
#Load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,RQAF6a0akMiot5lZZnMNNw,dalessandro-s-steaks-and-hoagies-philadelphia,Dalessandro’s Steaks & Hoagies,https://s3-media3.fl.yelpcdn.com/bphoto/jRs94o...,False,https://www.yelp.com/biz/dalessandro-s-steaks-...,3183,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 40.029494, 'longitude': -75.2059714}",[delivery],$$,"{'address1': '600 Wendover St', 'address2': ''...",+12154825407,(215) 482-5407,8111.519762
1,LM54ufrINJWoTN5imV8Etw,johns-roast-pork-philadelphia,John's Roast Pork,https://s3-media3.fl.yelpcdn.com/bphoto/qPyjRa...,False,https://www.yelp.com/biz/johns-roast-pork-phil...,1757,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 39.9210273851256, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '14 E Snyder Ave', 'address2': ''...",+12154631951,(215) 463-1951,9321.694746
2,-mIlmp5l4hKlp1tvHRdvTg,sonnys-famous-steaks-philadelphia,Sonny's Famous Steaks,https://s3-media2.fl.yelpcdn.com/bphoto/xtspCM...,False,https://www.yelp.com/biz/sonnys-famous-steaks-...,1893,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.0,"{'latitude': 39.94982923010674, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '228 Market St', 'address2': '', ...",+12156295760,(215) 629-5760,6264.888631
3,jxEMFqwDJXjCxmcm5t5jVQ,cleavers-philadelphia,Cleavers,https://s3-media1.fl.yelpcdn.com/bphoto/9dfyW_...,False,https://www.yelp.com/biz/cleavers-philadelphia...,1547,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.0,"{'latitude': 39.9513931274414, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '108 S 18th St', 'address2': '', ...",+12155153828,(215) 515-3828,7227.916750
4,q-zV08jt6U-q05SMEuQJAQ,tony-and-nicks-steaks-philadelphia,Tony & Nick's Steaks,https://s3-media1.fl.yelpcdn.com/bphoto/NW552n...,False,https://www.yelp.com/biz/tony-and-nicks-steaks...,2017,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 39.9141065085686, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '39 E Oregon Ave', 'address2': ''...",+12155515725,(215) 551-5725,10146.334733


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,4OGCFfm2vCnuPFRhezoYlg,chanks-grab-n-go-philadelphia-3,Chank's Grab-N-Go,https://s3-media1.fl.yelpcdn.com/bphoto/a1eoc9...,False,https://www.yelp.com/biz/chanks-grab-n-go-phil...,2,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",3.5,"{'latitude': 39.966144, 'longitude': -75.191074}","[delivery, pickup]",NaN,"{'address1': '3300 Fairmount Ave', 'address2':...",,,7426.259335
996,vXjCBd1_SG9FtJxJroU7TA,rosario-s-pizzeria-northeast-cheltenham,Rosario’s Pizzeria Northeast,https://s3-media2.fl.yelpcdn.com/bphoto/kVLosH...,False,https://www.yelp.com/biz/rosario-s-pizzeria-no...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 40.06900736126877, 'longitude': -...",[delivery],NaN,"{'address1': '576 Township Line Rd', 'address2...",+14452463178,(445) 246-3178,7648.209899
997,fGXhWKQV5YwjIjgP-DBNBw,liberty-bell-gastropub-bensalem,Liberty Bell Gastropub,https://s3-media2.fl.yelpcdn.com/bphoto/rKWMe_...,False,https://www.yelp.com/biz/liberty-bell-gastropu...,125,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 40.119244299997334, 'longitude': ...",[],$$,"{'address1': '2999 Street Rd', 'address2': '',...",+12675257566,(267) 525-7566,19137.710396
998,xkoMuRwlujZyQkijTtsmOg,sandwich-club-croydon,Sandwich Club,https://s3-media1.fl.yelpcdn.com/bphoto/CGsCGz...,False,https://www.yelp.com/biz/sandwich-club-croydon...,25,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 40.0942134, 'longitude': -74.9066...","[delivery, pickup]",$,"{'address1': '105 Newportville Rd', 'address2'...",+12157882807,(215) 788-2807,20670.264486
999,LiH_a3ZxwxKACzrYbzXXQQ,maple-glen-pizza-maple-glen,Maple Glen Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/COxKgv...,False,https://www.yelp.com/biz/maple-glen-pizza-mapl...,130,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,"{'latitude': 40.1795729, 'longitude': -75.1818...",[delivery],$$,"{'address1': '641 E Welsh Rd', 'address2': '',...",+12155427720,(215) 542-7720,20465.496925


In [29]:
#Check for duplicate ID's 
final_df.duplicated(subset='id').sum()

2

In [30]:
##Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [31]:
#Save the final results to a compressed csv
final_df.to_csv('Data/final_results_Philly_cheesesteak.csv.gz', compression='gzip',index=False)